This is Set 9: XGBoost with Polars (Exercises 81-90) of Polars Exercises: 💯 polars exercises You can find all the exercises and solutions on [GitHub](https://github.com/JustinKurland/polars_practice)

**Prerequisites**

*   The polars package should be upgraded to the latest version (v0.15.1+).
*   The sample dataset [1% sample of KDD Cup 2012 (CTR Prediction)](https://www.openml.org/search?type=data&sort=runs&id=1219&status=active) from openml will be used for the exercises.

In [1]:
!pip install polars
!pip install xgboost
!wget https://www.openml.org/data/get_csv/183151/kdd_ctr.csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-01-08 16:53:38--  https://www.openml.org/data/get_csv/183151/kdd_ctr.csv
Resolving www.openml.org (www.openml.org)... 131.155.11.11
Connecting to www.openml.org (www.openml.org)|131.155.11.11|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://api.openml.org/data/get_csv/183151/kdd_ctr.csv [following]
--2023-01-08 16:53:39--  https://api.openml.org/data/get_csv/183151/kdd_ctr.csv
Resolving api.openml.org (api.openml.org)... 131.155.11.11
Connecting to api.openml.org (api.openml.org)|131.155.11.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/plain]
Saving to: ‘kdd_ctr.csv.1’

kdd_ctr.csv.1           [         <=>        ]  27.50M  16.4MB/s    in 1.7s    

2023-01-08 16:53:41 (16.4 MB/s) - ‘kd

In [2]:
import polars as pl

data = pl.read_csv('kdd_ctr.csv')
data

click,impression,url_hash,ad_id,advertiser_id,depth,position,query_id,keyword_id,title_id,description_id,user_id
i64,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64
0,1,1.0710e19,8343295,11700,3,3,7702266,21264,27892,1559,0
1,1,1.7364e19,20017077,23798,1,1,93079,35498,4,36476,562934
0,1,8.9155e18,21348354,36654,1,1,10981,19975,36105,33292,11621116
0,1,4.4267e18,20366086,33280,3,3,0,5942,4057,4390,8778348
0,1,1.1573e19,6803526,10790,2,1,9881978,60593,25242,1679,12118311
1,1,2.8276e17,21186478,35793,2,1,163315,4871,3257,1153,2886008
0,1,8.8139e18,20886690,34840,2,2,316,543,2206,2888,7589739
0,1,3.8110e18,21367376,20667,3,2,2601439,118,9594,9705,579253
0,1,9.8068e18,21811752,37737,3,2,1631,333,841,2175,5277279


**Exercise 81: Initialize a `XGBoost` model**

In [3]:
import xgboost as xgb

model = xgb.XGBClassifier(objective='binary:logistic')
model

XGBClassifier()

**Exercise 82: Display the hyperparameters of xgboost model**

In [4]:
model.get_xgb_params()

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'learning_rate': 0.1,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'missing': None,
 'n_estimators': 100,
 'nthread': 1,
 'objective': 'binary:logistic',
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'seed': 0,
 'subsample': 1,
 'verbosity': 1}

**Exercise 83: Split `data` into `train_data` consisting of the first 75% of rows and `valid_data` consisting of the last 25% of rows**

In [5]:
data.shape

(399482, 12)

In [6]:
train_data = data.limit(round(data.height*.75))
train_data.shape

(299612, 12)

In [7]:
valid_data = data[(train_data.height):(data.height), :]
valid_data.shape

(99870, 12)

**Exercise 84: Assign the target variable `click` to `train_target` from `train_data` and `valid_target` from `valid_data` and drop the columns `url_hash` and `click` from `train_data` and `valid_data`**

In [8]:
train_target = train_data['click']
train_target.head()

click
i64
0
1
0
0
0
1
0
0
0


In [9]:
valid_target = valid_data['click']
valid_target.head()

click
i64
0
0
0
0
1
0
0
0
0


In [10]:
train_data = train_data.drop(["url_hash", "click"])
train_data.head()

impression,ad_id,advertiser_id,depth,position,query_id,keyword_id,title_id,description_id,user_id
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,8343295,11700,3,3,7702266,21264,27892,1559,0
1,20017077,23798,1,1,93079,35498,4,36476,562934
1,21348354,36654,1,1,10981,19975,36105,33292,11621116
1,20366086,33280,3,3,0,5942,4057,4390,8778348
1,6803526,10790,2,1,9881978,60593,25242,1679,12118311


In [11]:
valid_data = valid_data.drop(["url_hash", "click"])
valid_data.head()

impression,ad_id,advertiser_id,depth,position,query_id,keyword_id,title_id,description_id,user_id
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,21104237,29683,2,2,10689741,11495,43449,1704,11631003
1,21162526,1325,2,1,15,10,24,25,2220103
1,9027613,23808,2,1,2940,692,138,198,11428948
1,21866030,385,3,1,3801572,742504,2551939,1725632,1927769
1,10397715,27354,3,1,41805,15602,675726,315253,340926


**Exercise 85: Train the xgboost `model` using `train_data` and `train_target`**

In [12]:
model.fit(train_data.to_pandas(), train_target)
model

XGBClassifier()

**Exercise 86: Assign the feature importances of model to `model_fimp`**

In [13]:
model_fimp = pl.concat([pl.DataFrame(pl.Series(model.get_booster().feature_names).alias("Features")), 
                        pl.DataFrame(pl.Series(model.feature_importances_).alias('Importances'))], 
                        how = "horizontal")
model_fimp

Features,Importances
str,f32
"""impression""",0.156713
"""ad_id""",0.076684
"""advertiser_id""",0.067882
"""depth""",0.081832
"""position""",0.228727
"""query_id""",0.135929
"""keyword_id""",0.07408
"""title_id""",0.03829
"""description_id...",0.021889


**Exercise 87: Use `model` to predict on `valid_data` and assign the predictions to `preds_model`**

In [14]:
preds_model = pl.DataFrame(model.predict_proba(valid_data.to_pandas()))
names = ['false_prob','true_prob']
preds_model.columns = preds_model.columns[:0]+names
preds_model

false_prob,true_prob
f32,f32
0.875104,0.124896
0.69045,0.30955
0.707572,0.292429
0.528831,0.471169
0.740549,0.259451
0.826588,0.173412
0.971972,0.028028
0.88977,0.11023
0.534367,0.465633


**Exercise 88: Calculate the logloss of `preds_model` with `valid_target`**

In [15]:
preds = pl.concat([preds_model, pl.DataFrame(valid_target)], how = 'horizontal')
preds = preds.rename({"click": "true_label"})
preds

false_prob,true_prob,true_label
f32,f32,i64
0.875104,0.124896,0
0.69045,0.30955,0
0.707572,0.292429,0
0.528831,0.471169,0
0.740549,0.259451,1
0.826588,0.173412,0
0.971972,0.028028,0
0.88977,0.11023,0
0.534367,0.465633,0


In [16]:
import polars as pl

def logloss(df):
    """ Compute the logarithmic loss between the actual labels and the predicted probabilities.
    
    Parameters
    ----------
    df : polars.DataFrame
        A DataFrame with columns 'true_label', 'false_prob', and 'true_prob'.
        
    Returns
    -------
    polars.DataFrame
        A DataFrame with the logarithmic loss.
    """
    logloss = (((preds.select(pl.col(['true_label'])) *\
                 preds.select(pl.col(['true_prob']).log()) +\
               ((preds.select(pl.col(['true_label'])))-1) * ((preds.select(pl.col(['true_label'])))-1) *\
                 preds.select(pl.col(['false_prob']).log()))*-1)\
                .mean())
    
    logloss = logloss.select(pl.col(['true_label']).alias('logloss'))

    return logloss

In [17]:
logloss(preds)

logloss
f64
0.408759


**Exercise 89: Calculate the precision and recall of a new `label_model` that returns a crisp class (i.e, 0 or 1) rather than a pseudo-probability leveraging the `valid_target`**

In [18]:
label_model = pl.DataFrame(model.predict(valid_data.to_pandas()))
names = ['predicted_label']
label_model.columns = label_model.columns[:0]+names
label_model

label = pl.concat([label_model, pl.DataFrame(valid_target)], how = 'horizontal')
label = label.rename({"click": "true_label"})
label

predicted_label,true_label
i64,i64
0,0
0,0
0,0
0,0
0,1
0,0
0,0
0,0
0,0


In [19]:
import polars as pl

def precision_recall(df):
    """ Calculate precision and recall from a DataFrame with columns for true labels and predicted labels.
    
    Parameters
    ----------
    df : polars.DataFrame
        A DataFrame with columns 'true_label' and 'predicted_label'.
        
    Returns
    -------
    polars.DataFrame
        A DataFrame with the logarithmic loss.
    """
    true_positives  = (df.filter((pl.col("true_label") == 1) & (pl.col("predicted_label") == 1))).height
    false_positives = (df.filter((pl.col("true_label") == 0) & (pl.col("predicted_label") == 1))).height
    false_negatives = (df.filter((pl.col("true_label") == 1) & (pl.col("predicted_label") == 0))).height
    
    precision_recall = pl.DataFrame({'precision': true_positives / (true_positives + false_positives),\
            'recall': true_positives / (true_positives + false_negatives)})

    return precision_recall


In [20]:
precision_recall(label)

precision,recall
f64,f64
0.715556,0.0386


**Exercise 89: Calculate the F1 score from `label_model`**

In [21]:
import polars as pl

def f1_score(df):
    """ Calculate the F1 score from a DataFrame with columns for true labels and predicted labels.
    
    Parameters
    ----------
    df : polars.DataFrame
        A DataFrame with columns 'true_label' and 'predicted_label'.
        
    Returns
    -------
    polars.DataFrame
        A DataFrame with the logarithmic loss.
    """
    true_positives  = (df.filter((pl.col("true_label") == 1) & (pl.col("predicted_label") == 1))).height
    false_positives = (df.filter((pl.col("true_label") == 0) & (pl.col("predicted_label") == 1))).height
    false_negatives = (df.filter((pl.col("true_label") == 1) & (pl.col("predicted_label") == 0))).height
    
    precision = true_positives / (true_positives + false_positives)
    recall    = true_positives / (true_positives + false_negatives)

    F1 = pl.DataFrame({'F1': 2 * (precision * recall) / (precision + recall)})
    
    return F1

In [22]:
f1_score(label)

F1
f64
0.073248
